In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
import os
import typing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tqdm.notebook import tqdm
from statistics import mean,stdev
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
import random

from tcn import TCN
import wget
from getData import getData
from sklearn.metrics import *
from residual_momentum import resMomentum
import yfinance as yf

from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import concurrent.futures
import xgboost as xgb


In [63]:
stock_prices = pd.read_excel('data/halal_stocks.xlsx')
stock_prices = stock_prices.rename(columns={'Unnamed: 0':'Datetime'})
stock_prices['Datetime'] = pd.to_datetime(stock_prices['Datetime'])
stock_prices = stock_prices.set_index('Datetime')
stock_prices = stock_prices.dropna(axis=1,how='any')


In [64]:
lookbackperiod = 12
holdingperiod = 12
ranked_stocks,holding_returns,cumulative_returns = resMomentum(lookbackperiod,holdingperiod).getRankings(stock_prices)

[*********************100%%**********************]  1 of 1 completed


In [65]:
interval = '1d'
startdate = '2023-01-01'
stock_data = {}


In [66]:
for i in range(len(ranked_stocks)):
    stock_data[f'{ranked_stocks[i]}'] = yf.download(ranked_stocks[i],start=startdate,interval=interval)
    stock_data[f'{ranked_stocks[i]}'].index = pd.to_datetime(stock_data[f'{ranked_stocks[i]}'].index)
    
    stock_data[ranked_stocks[i]]['Tomorrow'] = stock_data[ranked_stocks[i]]["Close"].shift(-1)
    stock_data[ranked_stocks[i]]["Target"] = (stock_data[ranked_stocks[i]]["Tomorrow"] > stock_data[ranked_stocks[i]]["Close"]).astype(int)
    stock_data[ranked_stocks[i]] = stock_data[ranked_stocks[i]].iloc[:-1]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [83]:
import xgboost as xgb
from sklearn.metrics import precision_score


def prepare_data(dataframe):

    X = dataframe[['Open', 'High', 'Low', 'Close', 'Adj Close']]
    y = dataframe['Target']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

def train_model(dataframe):

    X_train, X_test, y_train, y_test = prepare_data(dataframe)

    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    #accuracy = precision_score(y_test,predictions)
    return model, accuracy


with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(train_model, df): name for name, df in stock_data.items()}
    for future in concurrent.futures.as_completed(futures):
        df_name = futures[future]
        try:
            model, accuracy = future.result()
            print(f"Model for {df_name} trained with accuracy: {accuracy}")
        except Exception as e:
            print(f"Error training model for {df_name}: {e}")




Model for FLWS trained with accuracy: 0.6274509803921569
Model for DDD trained with accuracy: 0.49019607843137253
Model for MMM trained with accuracy: 0.5098039215686274
Model for ANF trained with accuracy: 0.6078431372549019
Model for AAN trained with accuracy: 0.5490196078431373
Model for ABT trained with accuracy: 0.5490196078431373
Model for AAON trained with accuracy: 0.5098039215686274
Model for AOS trained with accuracy: 0.35294117647058826
Model for ABBV trained with accuracy: 0.47058823529411764
Model for ABM trained with accuracy: 0.49019607843137253
Model for ACIW trained with accuracy: 0.5882352941176471
Model for AYI trained with accuracy: 0.49019607843137253
Model for ACMR trained with accuracy: 0.47058823529411764
Model for GOLF trained with accuracy: 0.5490196078431373
Model for ADUS trained with accuracy: 0.45098039215686275
Model for ATGE trained with accuracy: 0.5490196078431373
Model for AEY trained with accuracy: 0.49019607843137253
Model for ADNT trained with accu

In [88]:
new_dict = stock_data

In [97]:
for i in range(len(ranked_stocks)):

    new_dict[f'{ranked_stocks[i]}'] = stock_data[f'{ranked_stocks[i]}']['Close']
    
    new_dict[f'{ranked_stocks[i]}'] = pd.DataFrame(new_dict[f'{ranked_stocks[i]}'])
    new_dict[f'{ranked_stocks[i]}'] = new_dict[f'{ranked_stocks[i]}'].rename(columns={'Close': ranked_stocks[i]})

In [99]:
import concurrent.futures
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler




def prepare_data_for_TCN(dataframe, look_back):
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataframe.values.reshape(-1, 1))

    
    X, y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i-look_back:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)

    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    return X, y, scaler

def build_TCN_model(input_shape):
    model = Sequential([
        TCN(input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_TCN_model(dataframe):
    look_back = 60  
    X, y, scaler = prepare_data_for_TCN(dataframe, look_back)
    model = build_TCN_model((X.shape[1], 1))
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)
    return model, scaler


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(train_TCN_model, df): name for name, df in new_dict.items()}
    for future in concurrent.futures.as_completed(futures):
        ticker_name = futures[future]
        try:
            model, scaler = future.result()
            print(f"Model for {ticker_name} trained successfully.")
            
        except Exception as e:
            print(f"Error training model for {ticker_name}: {e}")










Model for DDD trained successfully.
Model for MMM trained successfully.
Model for ABT trained successfully.
Model for AAN trained successfully.
Model for ANF trained successfully.
Model for FLWS trained successfully.
Model for ABBV trained successfully.
Model for ABM trained successfully.
Model for AOS trained successfully.
Model for AAON trained successfully.
Model for ACIW trained successfully.
Model for ACMR trained successfully.
Model for AYI trained successfully.
Model for GOLF trained successfully.
Model for ADUS trained successfully.
Model for AEY trained successfully.
Model for ADNT trained successfully.
Model for ADBE trained successfully.
Model for IE trained successfully.
Model for ATGE trained successfully.
